In [1]:
import socket

In [2]:
serverSocket = socket.socket()
server_address = ("192.168.1.6",3000)
serverSocket.bind(server_address)
serverSocket.listen(5)

In [ ]:
while True:
    
    print(f"Listening for clients on port {server_address[1]}")
    clientConnection, clientAddress = serverSocket.accept()
    
    print(f"Yessss! Got a connection from {clientAddress}")        
    data = clientConnection.recv(20).decode("utf-8")
        
    if(data == "hello"):
        print(f"User {clientAddress} sent {data}")
    else:
        print(f"Actually, user {clientAddress} didn't send hello but {data}")

Listening for clients on port 3000
Yessss! Got a connection from ('192.168.1.5', 36058)
User ('192.168.1.5', 36058) sent hello
Listening for clients on port 3000
